How is the pref score changing with noisy data (or smoother data)

In [28]:
import math
import pandas as pd
import altair as alt
import numpy as np

import sys, os, os.path

sys.path.append(os.path.expanduser("../src"))
sys.path.append(os.path.expanduser("../"))

from spinorama.compute_cea2034 import estimated_inroom
from spinorama.compute_scores import scores
from spinorama.graph import graph_params_default, graph_freq, graph_spinorama
from spinorama.load import filter_graphs, load_normalize
from generate_common import cache_load

df_all = cache_load(smoke_test=False)
print("{} loaded".format(len(df_all.keys())))

303 loaded


Test 2 strategies: add noise and smooth by average over a rolling window

In [29]:
def noise(df, amp):
    sdf = df.copy()
    l = len(sdf.Freq.values)
    for c in sdf.columns:
        if c != "Freq":
            sdf[c] += amp * (np.random.rand(l) - 0.5)
    return sdf


def smooth(df, amp):
    sdf = df.copy()
    l = len(sdf.Freq.values)
    for c in sdf.columns:
        if c != "Freq":
            sdf[c] = sdf[c].rolling(amp).mean()
    sdf.fillna(df)
    return sdf

In [30]:
def eval_score_noise(df_all, speaker, origin, version, amplitude):
    df_klippel = df_all[speaker][origin][version]
    spl_H = noise(df_klippel["SPL Horizontal_unmelted"], amplitude)
    spl_V = noise(df_klippel["SPL Vertical_unmelted"], amplitude)
    df_smoothed = load_normalize(filter_graphs(speaker, spl_H, spl_V))
    return scores(df_smoothed)["pref_score"]


def eval_score_smooth(df_all, speaker, origin, version, amplitude):
    df_klippel = df_all[speaker][origin][version]
    if amplitude == 0:
        spl_H = df_klippel["SPL Horizontal_unmelted"]
        spl_V = df_klippel["SPL Vertical_unmelted"]
    else:
        spl_H = smooth(df_klippel["SPL Horizontal_unmelted"], amplitude)
        spl_V = smooth(df_klippel["SPL Vertical_unmelted"], amplitude)
    df_smoothed = load_normalize(filter_graphs(speaker, spl_H, spl_V))
    return scores(df_smoothed)["pref_score"]

In [31]:
speakers = ["Adam S2V", "Emotiva Airmotiv B1+", "Genelec 8030C", "Dutch Dutch 8C"]
origins = ["ASR", "ASR", "ASR", "ErinsAudioCorner"]
versions = ["asr", "asr", "asr", "eac"]

In [32]:
range_noise = np.linspace(0, 3, 12)
range_smooth = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 30]

In [33]:
df_noise = pd.DataFrame({"Amplitude": range_noise})
df_smooth = pd.DataFrame({"Window": range_smooth})

for speaker, origin, version in zip(speakers, origins, versions):
    res_noise = [
        (a, eval_score_noise(df_all, speaker, origin, version, a)) for a in range_noise
    ]
    df_noise[speaker] = [j for i, j in res_noise]
    res_smooth = [
        (a, eval_score_smooth(df_all, speaker, origin, version, a))
        for a in range_smooth
    ]
    df_smooth[speaker] = [j for i, j in res_smooth]

In [34]:
chart_noise = (
    alt.Chart(df_noise.melt("Amplitude", var_name="Speakers", value_name="Score"))
    .mark_line()
    .encode(
        x=alt.X(
            "Amplitude:N", axis=alt.Axis(format=".1f", title="Noise amplitude (dB)")
        ),
        y=alt.Y("Score:Q", scale=alt.Scale(domain=[4, 9])),
        color=alt.Color("Speakers:N"),
    )
    .properties(width=400, title="Score v.s. Noise")
)
chart_smooth = (
    alt.Chart(df_smooth.melt("Window", var_name="Speakers", value_name="Score"))
    .mark_line()
    .encode(
        x=alt.X("Window:N", axis=alt.Axis(title="Size of the rolling window")),
        y=alt.Y("Score:Q", scale=alt.Scale(domain=[4, 9])),
        color=alt.Color("Speakers:N"),
    )
    .properties(width=400, title="Score v.s. Smoothing")
)

In [35]:
chart_noise | chart_smooth

alt.HConcatChart(...)

Can we estimates the smoothness of a curve?

In [69]:
res = []
for speaker, speaker_data in df_all.items():
    if len(speaker_data.keys()) == 1:
        continue
    for origin, measurements in speaker_data.items():
        version = list(df_all[speaker][origin].keys())[0]
        bv = None
        cea2034 = None
        if "CEA2034_unmelted" in df_all[speaker][origin][version].keys():
            cea2034 = df_all[speaker][origin][version]["CEA2034_unmelted"]
        elif "CEA2034" in df_all[speaker][origin][version].keys():
            cea2034_tmp = df_all[speaker][origin][version]["CEA2034"]
            try:
                cea2034 = (
                    cea2034_tmp.pivot(*cea2034_tmp)
                    .rename_axis(columns=None)
                    .reset_index()
                )
            except ValueError as ve:
                # print('{} pivot failed {}'.format(speaker, ve))
                continue
        if (
            cea2034 is not None
            and "On Axis" in cea2034.keys()
            and "Freq" in cea2034.keys()
        ):
            onaxis = cea2034["On Axis"]
            freq = cea2034["Freq"]
            n = len(freq)
            bv = np.mean(np.abs(np.gradient(onaxis)))
            bm = np.quantile(np.abs(np.gradient(onaxis)), 0.9)
            res.append((origin, bv, bm, speaker))

for (origin, bv, bm, speaker) in sorted(res, key=lambda a: a[1]):
    print("{:>25s} {:=5.3f} {:=5.3f} {:>50s}".format(origin, bv, bm, speaker))

         ErinsAudioCorner 0.006 0.010                                   Philharmonic BMR
         ErinsAudioCorner 0.007 0.011                                       Revel F226be
                Princeton 0.028 0.063                          Spendor Audio Systems SA1
                Princeton 0.034 0.067                                           KEF LS50
                Princeton 0.039 0.085                        Ascend Acoustics CBM-170 SE
                     Misc 0.080 0.170                                   Philharmonic BMR
              Vendors-JBL 0.083 0.200                                       JBL HDI-3600
                     Misc 0.087 0.196                                       Revel M126Be
         Vendors-Infinity 0.092 0.253                            Infinity Interlude IL10
                     Misc 0.096 0.204                                     Dutch Dutch 8C
            Vendors-Revel 0.101 0.188                                         Revel M105
            Vendors-R